In [1]:
%matplotlib inline

In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install moabb

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install braindecode

In [5]:
pip install torchinfo

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install torchvision 

Note: you may need to restart the kernel to use updated packages.


In [7]:
print(torch.__version__)

NameError: name 'torch' is not defined

In [ ]:
from braindecode.util import set_random_seeds, np_to_var, var_to_np
import matplotlib.pyplot as plt
from moabb.datasets import BNCI2014001, Cho2017, PhysionetMI
from moabb.paradigms import MotorImagery
import numpy as np
from numpy.random import RandomState
import pickle
import time
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

import mne

from util.Motor_Net9 import DpEEG_net
from util.utilfunc import get_balanced_batches
from util.preproc import plot_confusion_matrix

cuda = torch.cuda.is_available()
print('gpu: ', cuda)
device = 'cuda' if cuda else 'cpu'

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
rng = RandomState(seed)

In [ ]:
 mne.set_config('MNE_DATA', 'D:\EEG_Motor_Imagery\Dataset')
 mne.set_config('MNE_DATASETS_BNCI_PATH', 'D:\EEG_Motor_Imagery\Dataset')
 mne.set_config('MNE_DATASETS_EEGBCI_PATH', 'D:\EEG_Motor_Imagery\Dataset')
 mne.set_config('MNE_DATASETS_GIGADB_PATH', 'D:\EEG_Motor_Imagery\Dataset')
 print(mne.get_config())
# C:\Users\User\mne-data\

In [ ]:
start=time.time()

ds_src1 = Cho2017()
ds_src2 = PhysionetMI()
ds_tgt = BNCI2014001()

ds_src1_sub = [ i for i in range(1 , 32)] + [ i for i in range(33 , 46)] + [ i for i in range(47 , 49)] + [ i for i in range(50 , 53)]# last = 52

ds_src2_sub = [ i for i in range(1 , 110)] # last = 109

ds_tgt_sub = [ i for i in range(1 , 10)] # last = 9


fmin, fmax = 0, 60
raw = ds_tgt.get_data(subjects=[1])[1]['session_T']['run_1']
tgt_channels = raw.pick_types(eeg=True).ch_names
sfreq = 250.
prgm_2classes = MotorImagery(n_classes=2, channels=tgt_channels, resample=sfreq, fmin=fmin, fmax=fmax)
prgm_4classes = MotorImagery(n_classes=4, channels=tgt_channels, resample=sfreq, fmin=fmin, fmax=fmax)

X_src1, label_src1, m_src1 = prgm_2classes.get_data(dataset=ds_src1, subjects=ds_src1_sub)
print('Cho2017 done')
end = time.time()
print('time is {} min'.format(int((end-start)/60)))

X_src2, label_src2, m_src2 = prgm_4classes.get_data(dataset=ds_src2, subjects=ds_src2_sub)
print('PhysionetMI done')
end = time.time()
print('time is {} min'.format(int((end-start)/60)))
X_tgt, label_tgt, m_tgt = prgm_4classes.get_data(dataset=ds_tgt, subjects=ds_tgt_sub)
print('BNCI2014001 done')
end = time.time()
print('time is {} min'.format(int((end-start)/60)))

print("First source dataset has {} trials with {} electrodes and {} time samples".format(*X_src1.shape))
print("Second source dataset has {} trials with {} electrodes and {} time samples".format(*X_src2.shape))
print("Target dataset has {} trials with {} electrodes and {} time samples".format(*X_tgt.shape))

print ("\nSource dataset 1 include labels: {}".format(np.unique(label_src1)))
print ("Source dataset 2 include labels: {}".format(np.unique(label_src2)))
print ("Target dataset 1 include labels: {}".format(np.unique(label_tgt)))

In [ ]:
def relabel(l):
    if l == 'left_hand': return 0
    elif l == 'right_hand': return 1
    else: return 2


y_src1 = np.array([relabel(l) for l in label_src1])
y_src2 = np.array([relabel(l) for l in label_src2])
y_tgt = np.array([relabel(l) for l in label_tgt])

print("Only right-/left-hand labels are used and first source dataset does not have other labels:")
print(np.unique(y_src1), np.unique(y_src2), np.unique(y_tgt))

In [ ]:
# print(X_tgt.shape)
valid_size = int(X_tgt.shape[0]*.2)
test_size = int(X_tgt.shape[0]*.2)
# print(test_size)
train_size = X_tgt.shape[0] - (test_size + valid_size)

In [ ]:
window_size = min(X_src1.shape[2], X_src2.shape[2], X_tgt.shape[2])

X_train = np.concatenate((X_src1[:, :, :window_size], X_src2[:, :, :window_size], X_tgt[:train_size, :, :window_size]))
y_train = np.concatenate((y_src1, y_src2, y_tgt[:train_size]))

X_val = X_tgt[train_size:(train_size+valid_size), :, :window_size]
y_val = y_tgt[train_size:(train_size+valid_size)]

X_test = X_tgt[(train_size+valid_size):, :, :window_size]
y_test = y_tgt[(train_size+valid_size):]

print("Train:  there are {} trials with {} electrodes and {} time samples".format(*X_train.shape))
print("\nValidation: there are {} trials with {} electrodes and {} time samples".format(*X_val.shape))
print("\nTest: there are {} trials with {} electrodes and {} time samples".format(*X_test.shape))

In [ ]:
class TrainObject(object):
    def __init__(self, X, y):
        assert len(X) == len(y)
        # Normalised, you could choose other normalisation strategy
        mean = np.mean(X,axis=1,keepdims=True)
        # here normalise across channels as an example, unlike the in the sleep kit
        std = np.std(X, axis=1, keepdims=True)
        X = (X - mean) / std
        # we scale it to 1000 as a better training scale of the shallow CNN
        # according to the orignal work of the paper referenced above
        self.X = X.astype(np.float32) * 1e3
        self.y = y.astype(np.int64)

train_set = TrainObject(X_train, y=y_train)
valid_set = TrainObject(X_val, y=y_val)
test_set = TrainObject(X_test, y=y_test)

In [ ]:
print(X_train.shape)

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

# Hide the Configuration and Warnings
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

import random
import numpy as np
import tensorflow as tf
from Models.DatasetAPI.DataLoader import DatasetLoader
from Models.Network.Thin_ResNet import Thin_ResNet
from Models.Loss_Function.Loss import loss
from Models.Evaluation_Metrics.Metrics import evaluation

In [ ]:
import tensorflow as tf
from Models.Initialize_Variables.Initialize import *


def Thin_ResNet(Input, keep_prob):
    '''
    Args:
        Input: The reshaped input EEG signals
        keep_prob: The Keep probability of Dropout
    Returns:
        prediction: Final prediction of Thin ResNet Model
    '''

    # Input reshaped EEG signals: shape 4096 --> 64 X 64
    x_Reshape = tf.reshape(tensor=Input, shape=[-1, 64, 64, 1])

    # First Residual Block
    # First Convolutional Layer
    W_conv1 = weight_variable([1, 1, 1, 48])
    b_conv1 = bias_variable([48])
    h_conv1 = tf.nn.conv2d(x_Reshape, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
    h_conv1_BN = tf.layers.batch_normalization(h_conv1, training=True)
    h_conv1_Acti = tf.nn.leaky_relu(h_conv1_BN)
    h_conv1_drop = tf.nn.dropout(h_conv1_Acti, keep_prob, noise_shape=[tf.shape(h_conv1_Acti)[0], 1, 1, tf.shape(h_conv1_Acti)[3]])

    # Second Convolutional Layer
    W_conv2 = weight_variable([3, 3, 48, 48])
    b_conv2 = bias_variable([48])
    h_conv2 = tf.nn.conv2d(h_conv1_drop, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2
    h_conv2_BN = tf.layers.batch_normalization(h_conv2, training=True)
    h_conv2_Acti = tf.nn.leaky_relu(h_conv2_BN)
    h_conv2_drop = tf.nn.dropout(h_conv2_Acti, keep_prob, noise_shape=[tf.shape(h_conv2_Acti)[0], 1, 1, tf.shape(h_conv2_Acti)[3]])

    # Third Convolutional Layer
    W_conv3 = weight_variable([1, 1, 48, 96])
    b_conv3 = bias_variable([96])
    h_conv3 = tf.nn.conv2d(h_conv2_drop, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3
    h_conv3_BN = tf.layers.batch_normalization(h_conv3, training=True)
    h_conv3_res = tf.concat([h_conv3_BN, x_Reshape], axis=3)  # 97 feature maps now == 96 + 1
    h_conv3_Acti = tf.nn.leaky_relu(h_conv3_res)
    h_conv3_drop = tf.nn.dropout(h_conv3_Acti, keep_prob, noise_shape=[tf.shape(h_conv3_Acti)[0], 1, 1, tf.shape(h_conv3_Acti)[3]])

    # First Max Pooling Layer: shape 64 X 64 --> 32 X 32
    h_pool1 = tf.nn.max_pool(h_conv3_drop, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Second Residual Block
    # Fourth Convolutional Layer
    W_conv4 = weight_variable([1, 1, 97, 96])
    b_conv4 = bias_variable([96])
    h_conv4 = tf.nn.conv2d(h_pool1, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4
    h_conv4_BN = tf.layers.batch_normalization(h_conv4, training=True)
    h_conv4_Acti = tf.nn.leaky_relu(h_conv4_BN)
    h_conv4_drop = tf.nn.dropout(h_conv4_Acti, keep_prob, noise_shape=[tf.shape(h_conv4_Acti)[0], 1, 1, tf.shape(h_conv4_Acti)[3]])

    # Fifth Convolutional Layer
    W_conv5 = weight_variable([3, 3, 96, 96])
    b_conv5 = bias_variable([96])
    h_conv5 = tf.nn.conv2d(h_conv4_drop, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5
    h_conv5_BN = tf.layers.batch_normalization(h_conv5, training=True)
    h_conv5_Acti = tf.nn.leaky_relu(h_conv5_BN)
    h_conv5_drop = tf.nn.dropout(h_conv5_Acti, keep_prob, noise_shape=[tf.shape(h_conv5_Acti)[0], 1, 1, tf.shape(h_conv5_Acti)[3]])

    # Sixth Convolutional Layer
    W_conv6 = weight_variable([1, 1, 96, 128])
    b_conv6 = bias_variable([128])
    h_conv6 = tf.nn.conv2d(h_conv5_drop, W_conv6, strides=[1, 1, 1, 1], padding='SAME') + b_conv6
    h_conv6_BN = tf.layers.batch_normalization(h_conv6, training=True)
    h_conv6_res = tf.concat([h_conv6_BN, h_pool1], axis=3)  # 225 feature maps now == 97 + 128
    h_conv6_Acti = tf.nn.leaky_relu(h_conv6_res)
    h_conv6_drop = tf.nn.dropout(h_conv6_Acti, keep_prob, noise_shape=[tf.shape(h_conv6_Acti)[0], 1, 1, tf.shape(h_conv6_Acti)[3]])

    # Second Max Pooling Layer: shape 32 X 32 --> 16 X 16
    h_pool2 = tf.nn.max_pool(h_conv6_drop, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Third Residual Block
    # Seventh Convolutional Layer
    W_conv7 = weight_variable([1, 1, 225, 128])
    b_conv7 = bias_variable([128])
    h_conv7 = tf.nn.conv2d(h_pool2, W_conv7, strides=[1, 1, 1, 1], padding='SAME') + b_conv7
    h_conv7_BN = tf.layers.batch_normalization(h_conv7, training=True)
    h_conv7_Acti = tf.nn.leaky_relu(h_conv7_BN)
    h_conv7_drop = tf.nn.dropout(h_conv7_Acti, keep_prob, noise_shape=[tf.shape(h_conv7_Acti)[0], 1, 1, tf.shape(h_conv7_Acti)[3]])

    # Eighth Convolutional Layer
    W_conv8 = weight_variable([3, 3, 128, 128])
    b_conv8 = bias_variable([128])
    h_conv8 = tf.nn.conv2d(h_conv7_drop, W_conv8, strides=[1, 1, 1, 1], padding='SAME') + b_conv8
    h_conv8_BN = tf.layers.batch_normalization(h_conv8, training=True)
    h_conv8_Acti = tf.nn.leaky_relu(h_conv8_BN)
    h_conv8_drop = tf.nn.dropout(h_conv8_Acti, keep_prob, noise_shape=[tf.shape(h_conv8_Acti)[0], 1, 1, tf.shape(h_conv8_Acti)[3]])

    # Ninth Convolutional Layer
    W_conv9 = weight_variable([1, 1, 128, 256])
    b_conv9 = bias_variable([256])
    h_conv9 = tf.nn.conv2d(h_conv8_drop, W_conv9, strides=[1, 1, 1, 1], padding='SAME') + b_conv9
    h_conv9_BN = tf.layers.batch_normalization(h_conv9, training=True)
    h_conv9_res = tf.concat([h_conv9_BN, h_pool2], axis=3)  # 481 feature maps now == 225 + 256
    h_conv9_Acti = tf.nn.leaky_relu(h_conv9_res)
    h_conv9_drop = tf.nn.dropout(h_conv9_Acti, keep_prob, noise_shape=[tf.shape(h_conv9_Acti)[0], 1, 1, tf.shape(h_conv9_Acti)[3]])

    # Third Max Pooling Layer
    h_pool3 = tf.nn.max_pool(h_conv9_drop, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Flatten Layer
    h_pool6_flat = tf.reshape(h_pool3, [-1, 8 * 8 * 481])

    # First Fully Connected Layer
    W_fc1 = weight_variable([8 * 8 * 481, 512])
    b_fc1 = bias_variable([512])
    h_fc1 = tf.matmul(h_pool6_flat, W_fc1) + b_fc1
    h_fc1_BN = tf.layers.batch_normalization(h_fc1, training=True)
    h_fc1_Acti = tf.nn.leaky_relu(h_fc1_BN)
    h_fc1_drop = tf.nn.dropout(h_fc1_Acti, keep_prob)

    # Second Fully Connected Layer
    W_fc2 = weight_variable([512, 4])
    b_fc2 = bias_variable([4])
    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    return prediction

In [ ]:
# in_chans = X_train.shape[1]
X_train = tf.placeholder(tf.float32, [None, 4096])
y_train = tf.placeholder(tf.float32, [None, 4])
keep_prob = tf.placeholder(tf.float32)

model = CNN(Input=X_train, keep_prob=keep_prob)

if cuda:
    model.cuda()
    
optimizer = optim.AdamW(model.parameters(), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, 'min', factor = 0.5, patience = 5, min_lr = 0.00001, verbose = True)

In [ ]:
# summary(model, (70, 22, 750, 1))

In [ ]:
batch_size = 30

total_epoch = -1
Tlosses, Taccuracies = [], []
Vlosses, Vaccuracies = [], []
highest_acc = 0

savename = "Model/cnn_model_mi1.pth"

start=time.time()
for i_epoch in range(100):
    total_epoch += 1
    # Randomize batches ids and get iterater 'i_trials_in_batch'
    i_trials_in_batch = get_balanced_batches(len(train_set.X), rng, shuffle=True,
                                             batch_size=batch_size)
    # Set model to training mode
    model.train()
    for i_trials in i_trials_in_batch:
        # Have to add empty fourth dimension to X for training
        batch_X = train_set.X[i_trials][:, :, :, None]
        batch_y = train_set.y[i_trials]
        # convert from nparray to torch tensor
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        # Compute outputs of the network
        outputs = model(net_in)
        # Compute the loss
        loss = F.nll_loss(outputs, net_target)
        # Do the backpropagation
        loss.backward()
        # Update parameters with the optimizer
        optimizer.step()
    # Set model to evaluation mode
    model.eval()
    print("Epoch {:d}".format(total_epoch))
    average_acc = []
    average_loss = []
    
    # Here we compute training accuracy and validation accuracy of current model
    for setname, dataset in (('Train', train_set), ('Valid', valid_set)):
        i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                                 batch_size=batch_size)
        outputs=None
        for i_trials in i_trials_in_batch:
            batch_X = dataset.X[i_trials][:, :, :, None]
            batch_y = dataset.y[i_trials]
            net_in = np_to_var(batch_X)
            if cuda:
                net_in = net_in.cuda()
            toutputs = model(net_in)
            if outputs is None:
                temp = toutputs.cpu()
                outputs = temp.detach().numpy()
            else:
                temp = toutputs.cpu()
                outputs = np.concatenate((outputs,temp.detach().numpy()))
        net_target = np_to_var(dataset.y)
        loss = F.nll_loss(torch.from_numpy(outputs), net_target)
        print("{:6s} Loss: {:.5f}".format(
            setname, float(var_to_np(loss))))
        predicted_labels = np.argmax((outputs), axis=1)
        accuracy = np.mean(dataset.y  == predicted_labels)
        
        print("{:6s} Accuracy: {:.1f}%".format(setname, accuracy * 100))
        if setname == 'Train':
            Tlosses.append(loss)
            Taccuracies.append(accuracy)
            current_Tacc=accuracy
        elif setname == 'Valid':
            Vlosses.append(loss)
            Vaccuracies.append(accuracy)
            scheduler.step(loss)
            if accuracy>=highest_acc:
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict()
                }, savename)

                highest_acc=accuracy
                print('model saved')
                plot_confusion_matrix(dataset.y, predicted_labels, 
                                      classes=['LH', 'RH', 'Other'], normalize=True,
                                      title='Validation confusion matrix')
                plt.show()
        else:
            average_acc.append(accuracy)
            average_loss.append(accuracy)
end = time.time()

print('time is {} min'.format(int((end-start)/60)))

Shallow_net-> 100 epoches, batch_size = 100, train_acc = 58.3% and valid_acc = 61.8% and test_acc = 59.3%

Motor_net2-> 100 epoches, batch_size = 100, train_acc = 74.1% and valid_acc = 66.1% and test_acc = 64.1%

Motor_net3-> 100 epoches, batch_size = 50, train_acc = 66.7% and valid_acc = 64.7% and test_acc =62.6 %

Motor_net3_new-> 100 epoches, batch_size = 30, train_acc = 72.7% and valid_acc = 66.1% and test_acc =66.6 %

Motor_net4-> 78 epoches, batch_size = 20, train_acc = 68.0% and valid_acc = 57.0% and test_acc =55.1 %

Motor_net7-> 100 epoches, batch_size = 30, train_acc = 49.8% and valid_acc = 51.2% and test_acc = 46.1 %

Motor_net8-> 100 epoches, batch_size = 30, train_acc = 78.8 % and valid_acc = 67.5 % and test_acc = 69.8 %

Motor_net9-> 100 epoches, batch_size = 30, train_acc = 81.5 % and valid_acc = 68.2 % and test_acc = 68.9 %

In [ ]:
t = np.arange(0.0, len(Tlosses), 1)+1
plt.plot(t, Tlosses, 'r', t, Vlosses, 'y')
plt.legend(('Training loss', 'validation loss'))
plt.show()

plt.plot(t, Taccuracies, 'r', t, Vaccuracies, 'y')
plt.legend(('Training accuracy', 'Validation accuracy'))
plt.show()

In [ ]:
model = DpEEG_net()
savename = "Model/cnn_model_mi.pth"
if cuda:
    model.cuda()
    
checkpoint = torch.load(savename)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc, average_loss = [], []
setname = 'testset'
dataset = test_set

i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                         batch_size=30)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
    batch_X = dataset.X[i_trials][:, :, :, None]
    batch_y = dataset.y[i_trials]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp = toutputs.cpu()
        outputs = temp.detach().numpy()
    else:
        temp = toutputs.cpu()
        outputs = np.concatenate((outputs,temp.detach().numpy()))

net_target = np_to_var(dataset.y)
loss = F.nll_loss(torch.from_numpy(outputs), net_target)
print("{:6s} Loss: {:.5f}".format(setname, float(var_to_np(loss))))
predicted_labels = np.argmax((outputs), axis=1)
accuracy = np.mean(dataset.y  == predicted_labels)

print("{:6s} Accuracy: {:.1f}%".format(setname, accuracy * 100))
plot_confusion_matrix(dataset.y, predicted_labels, 
                      classes=['LH','RH','Other'], normalize=True,
                      title='Validation confusion matrix')
plt.show()